In [ ]:
!pip install mediapipe opencv-python tensorflow tensorflow-lite

In [51]:
import cv2
import mediapipe as mp
import tensorflow  as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import json




# ... rest of your code ...

# Segundo a pesquisa a biblioteca Face Recognitio (Dlib)  a mmelhro par o projeto


test_image = 'class.jpg'



In [48]:
import tensorflow as tf

# Caminho para o diretório onde está o modelo
model_dir = 'model'

# Carregar o modelo
model = tf.saved_model.load(model_dir)

# Verifique a assinatura do modelo
print(model.signatures)



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


_SignatureMap({})


In [49]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Salvar o modelo TensorFlow Lite
with open("modelo_reconhecimento_facial.tflite", "wb") as f:
    f.write(tflite_model)

# Inicializar MediaPipe para detecção de rostos
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# Carregar o modelo TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path="seu_modelo_de_embeddings.tflite")
interpreter.allocate_tensors()

# Função para obter embeddings
def extrair_embedding(face_image):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_data = np.expand_dims(face_image, axis=0).astype(np.float32) / 255.0
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    return interpreter.get_tensor(output_details[0]['index'])[0]

# Função principal para detecção de rostos e extração de embeddings
def reconhecer_presenca(imagem):
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        resultados = face_detection.process(imagem_rgb)

        embeddings_faces = []
        if resultados.detections:
            for deteccao in resultados.detections:
                bbox = deteccao.location_data.relative_bounding_box
                h, w, _ = imagem.shape
                x, y, largura, altura = int(bbox.xmin * w), int(bbox.ymin * h), int(bbox.width * w), int(bbox.height * h)
                face = imagem[y:y + altura, x:x + largura]
                face_resized = cv2.resize(face, (160, 160))  # Ajuste o tamanho conforme o modelo
                embeddings_faces.append(extrair_embedding(face_resized))

        return embeddings_faces


INFO:tensorflow:Assets written to: C:\Users\Erick\AppData\Local\Temp\tmpuq12jejc\assets


INFO:tensorflow:Assets written to: C:\Users\Erick\AppData\Local\Temp\tmpuq12jejc\assets


ConverterError: Could not translate MLIR to FlatBuffer.

In [55]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For static images:
IMAGE_FILES = [test_image]
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      # print('face_landmarks:', face_landmarks)
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
      
    cv2.imwrite('annotated_image' + str(idx) + '.png', annotated_image)

In [52]:

imagem = cv2.imread(test_image)
embeddings = reconhecer_presenca(imagem)
print("Embeddings extraídos:", embeddings)

NameError: name 'reconhecer_presenca' is not defined

In [ ]:
# Ler https://docs.ultralytics.com/quickstart/#use-ultralytics-with-python

# Testando eficiência de cada modelo


In [ ]:
#Modelo v8s
model = YOLO("yolov8s.pt") # load the model


#results = model.train(data="coco128.yaml", epochs=5)

results = model(test_image)

In [ ]:
# Usando o Modelo 11
model = YOLO("yolo11n.pt")  # load an official model
#model = YOLO("path/to/best.pt")  # load a custom model

# Predict with the model
results = model(test_image)  #predict on an image
#results

In [ ]:
# Teste para ver se reconhece uma escova de dentes

model = YOLO("yolo11n.pt")  # load an official model

#results = model("./tooth.png")  # predict on an image
#results

# Detecção de pessoas


In [ ]:
# importing the cv2 library
import cv2

# loading the haar case algorithm file into alg variable
alg = "/content/drive/MyDrive/Colab Notebooks/haarcascade_frontalface_default.xml"
# passing the algorithm to OpenCV
haar_cascade = cv2.CascadeClassifier(alg)
# reading the image
img = cv2.imread(test_image, 0)
# creating a black and white version of the image
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
# detecting the faces
faces = haar_cascade.detectMultiScale(
    gray_img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100)
)

i = 0
# for each face detected
for x, y, w, h in faces:
    # crop the image to select only the face
    cropped_image = img[y : y + h, x : x + w]
    # loading the target image path into target_file_name variable  - replace <INSERT YOUR TARGET IMAGE NAME HERE> with the path to your target image
    target_file_name = 'stored-faces/' + str(i) + '.jpg'
    cv2.imwrite(
        target_file_name,
        cropped_image,
    )
    i = i + 1;

In [ ]:
import numpy as np

# connecting to the database - replace the SERVICE URI with the service URI
conn = psycopg2.connect("<SERVICE_URI>")

for filename in os.listdir("stored-faces"):
    # opening the image
    img = Image.open("stored-faces/" + filename)
    # loading the `imgbeddings`
    ibed = imgbeddings()
    # calculating the embeddings
    embedding = ibed.to_embeddings(img)
    cur = conn.cursor()
    cur.execute("INSERT INTO pictures values (%s,%s)", (filename, embedding[0].tolist()))
    print(filename)
conn.commit()

In [ ]:

model = YOLO('yolov8s.pt')

# Carrega a imagem usando OpenCV
img = cv2.imread(test_image)
results = model(img)

after_img = results[0].plot()  # Desenha as caixas na imagem original
plt.imshow(cv2.cvtColor(after_img, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Remove os eixos
plt.show()


In [ ]:

# Lista de imagens de referência
reference_images = [
    ('/content/drive/MyDrive/Colab Notebooks/Erick', 'Erick'),
    ('/content/drive/MyDrive/Colab Notebooks/Plank', 'Amigo'),
    ('/content/drive/MyDrive/Colab Notebooks/Einstein', 'Mule')
]

# Verifica se a imagem foi carregada corretamente
if img is None:
    print("Erro ao carregar a imagem de teste.")
else:
    face_found = False

    # Loop para comparar a imagem de teste com as imagens de referência
    for path, name in reference_images:
        try:
            # Realiza a busca utilizando DeepFace
            result = DeepFace.find(img_path=test_image, db_path=path, model_name='VGG-Face')

            # Verifique o formato do retorno
            if not result:
                print(f"Nenhum resultado encontrado no diretório {path}.")
                continue

            # Iterate through the results and check if a match is found
            for item in result:
                # Aqui, pode ser necessário ajustar a verificação dependendo do formato do retorno
                if 'verified' in item and item['verified'] is True:
                    face_found = True
                    print(f'Rosto de {name} encontrado na imagem.')
                    break  # Sai do loop interno se uma correspondência for encontrada

            if face_found:
                break  # Sai do loop externo se uma correspondência for encontrada

        except Exception as e:
            print(f"Erro ao processar a pasta {path}: {e}")

    if not face_found:
        print("Nenhum rosto conhecido foi identificado na imagem.")

    # Exibe a imagem analisada
    #img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.show()


In [ ]:
from deepface import DeepFace

# Caminho da imagem de teste
img = '/content/drive/MyDrive/Colab Notebooks/20241113_154643.jpg'  # Substitua pelo caminho correto da imagem de teste

# Caminho da imagem de referência (imagem de Erick ou qualquer outra pessoa)
db = '/content/drive/MyDrive/Colab Notebooks/Erick/image.png'  # Substitua pela imagem de referência

# Realiza a verificação entre a imagem de teste e a referência
result = DeepFace.verify(img1_path=img, img2_path=db, model_name='VGG-Face')

# Exibe o resultado
print(result)
